In [1]:
TOTAL_PAGE_NUMBER = 50
KEYWORDS = [ 'python', '.net','数据挖掘','大数据','数据分析','推荐算法','机器学习','.net core']
ADDRESS = ['北京', ]
basic_url = 'http://sou.zhaopin.com/jobs/searchresult.ashx?'

paras = {'jl': '北京','kw': '数据挖掘','p': '1'}

In [2]:
from datetime import datetime
from urllib.parse import urlencode
import requests
from bs4 import BeautifulSoup
import time
from itertools import product
import random
import re

In [3]:
def download(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0'}
    response = requests.get(url, headers=headers)
    delay = random.randint(0, 2)
    time.sleep(delay)  # 设置等待时间，防止网站反爬虫
    return response.text

In [4]:
url = basic_url + urlencode(paras)

In [138]:
html = download(url)

In [8]:
def get_content(html):
    # 记录保存日期
    date = datetime.now().date()
    date = datetime.strftime(date, '%Y-%m-%d')  # 转变成str

    soup = BeautifulSoup(html, 'lxml')
    body = soup.body
    data_main = body.find('div', {'class': 'newlist_list_content'})

    if data_main:
        tables = data_main.find_all('table')

        for i, table_info in enumerate(tables):
            if i == 0:
                continue
            tds = table_info.find('tr').find_all('td')
            zwmc = tds[0].find('a').get_text()  # 职位名称
            zw_link = tds[0].find('a').get('href')  # 职位链接
            fkl = tds[1].find('span').get_text()  # 反馈率
            gsmc = tds[2].find('a').get_text()  # 公司名称
            zwyx = tds[3].get_text()  # 职位月薪
            gzdd = tds[4].get_text()  # 工作地点
            gbsj = tds[5].find('span').get_text()  # 发布日期

            tr_brief_spans = table_info.find(
                'tr', {'class': 'newlist_tr_detail'}).find_all('span')

            gsdd = ""
            gsxz = ""
            gsgm = ""
            gzjy = ""
            xlyq = ""
            zwyx_1 = ""
            for list in tr_brief_spans:
                if len(list.get_text().split('：')) >= 2:
                    if list.get_text().split('：')[0] == "地点":
                        gsdd = list.get_text().split('：')[1]  # 公司地点
                    elif list.get_text().split('：')[0] == "公司性质":
                        gsxz = list.get_text().split('：')[1]  # 公司性质
                    elif list.get_text().split('：')[0] == "公司规模":
                        gsgm = list.get_text().split('：')[1]  # 公司规模
                    elif list.get_text().split('：')[0] == "经验":
                        gzjy = list.get_text().split('：')[1]  # 工作经验
                    elif list.get_text().split('：')[0] == "学历":
                        xlyq = list.get_text().split('：')[1]  # 学历要求
                    elif list.get_text().split('：')[0] == "职位月薪":
                        zwyx_1 = list.get_text().split('：')[1]  # 职位月薪(元/月)
                    else:
                        break


            # 用生成器获取信息
            yield {'职位名称': zwmc,  # 职位名称
                   '反馈率': fkl,  # 反馈率
                   '公司名称': gsmc,  # 公司名称
                   '职位月薪': zwyx,  # 职位月薪
                   '工作地点': gzdd,  # 工作地点
                   '公布时间': gbsj,  # 公布时间
                   '职位链接': zw_link,  # 网页链接
                   '保存时间': date,  # 记录信息保存的日期
                   '公司地点': gsdd,  # 公司地点
                   '公司性质': gsxz,  # 公司性质
                   '公司规模': gsgm,  # 公司规模
                   '工作经验': gzjy,  # 工作经验
                   '学历要求': xlyq,  # 学历要求
                   '职业薪水': zwyx_1,  # 职位月薪(元/月)
                   }


In [89]:
def get_between_re(start,end):
    return '(?<={0}).*(?={1})'.format(start,end)

In [79]:
def get_search_str(match_re):
    if match_re:
        return match_re.group(0)
    return ''

In [156]:
def get_detal(html):
    soup = BeautifulSoup(html, 'lxml')
    body = soup.body
    data_main = body.find('div', {'class': 'tab-inner-cont'})
    if data_main:
        text = data_main.get_text()
        to_del_re = '\r|\n|\t| |查看职位地图'
        text_clear = re.sub(to_del_re,'',text)
#         print(text_clear)
        first_re = '((职位要求(：|:))|(职位描述(：|:))|(岗位职责(：|:))|(岗位描述(：|:))|(工作职责(：|:)))'
        second_re = '((岗位要求(：|:))|(职位要求(：|:))|(任职要求(：|:))|(任职资格(：|:))|(工作要求(：|:)))'
        third_re = '(工作地址(：|:))'
        responsibilities_search = re.search(get_between_re(first_re,second_re),text_clear)
        if '工作地址' in text_clear:
            requirements_search = re.search(get_between_re(second_re,third_re),text_clear)
        else:
            requirements_search = re.search('(?<={0}).*'.format(second_re),text_clear)
        addr_search = re.search('(?<={0}).*'.format(third_re),text_clear)
        responsibilities = get_search_str(responsibilities_search) #岗位职责
        requirements = get_search_str(requirements_search) #技能要求
        addr = get_search_str(addr_search) #公司地址
        return(responsibilities,requirements,addr)
                

In [115]:
url1 = 'http://jobs.zhaopin.com/120072290295717.htm'
html1 = download(url1)


In [ ]:
data = get_content(html)
for each in data:
    for k,v in each.items():
        if k == '职位链接':
            print(v)
            html1 = download(v)
            data1 = get_detal(html1)
            for s in data1:
                print(s)
            print('\r')